In [111]:
import tensorflow as tf
tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                          kernel_initializer=he_init, name="hidden1")

In [107]:

def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

def elu(z, alpha=1):
    return np.where(z < 0, alpha * (np.exp(z) - 1), z)


In [110]:


import numpy as np
tf.reset_default_graph()
import warnings
warnings.filterwarnings("ignore")


n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")


#tf.nn.relu, tf.nn.elu, tf.nn.selu
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.leaky_relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0:
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.98 Validation accuracy: 0.905


KeyboardInterrupt: 

In [15]:
tf.reset_default_graph()
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')
y = tf.placeholder(tf.int32, shape=(None), name="y")



# hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
# bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
# bn1_act = tf.nn.elu(bn1)

# hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
# bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
# bn2_act = tf.nn.elu(bn2)

# logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
# logits = tf.layers.batch_normalization(logits_before_bn, training=training,
#                                        momentum=0.9)

#------another way using partial--------------------

from functools import partial

# my_batch_norm_layer = partial(tf.layers.batch_normalization,
#                               training=training, momentum=0.9)

# hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
# bn1 = my_batch_norm_layer(hidden1)
# bn1_act = tf.nn.elu(bn1)

# hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
# bn2 = my_batch_norm_layer(hidden2)
# bn2_act = tf.nn.elu(bn2)

# logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
# logits = my_batch_norm_layer(logits_before_bn)

#--------------------------------------#

batch_norm_momentum = 0.9
with tf.name_scope("dnn"):
    he_init = tf.variance_scaling_initializer()

    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)

    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200


# tf.GraphKeys.UPDATE_OPS is a collection of these variables and if you put it in the tf.control_dependencies 
# block, these Ops will get executed before the training op is run.
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")



In [114]:
#####gradient clippng
tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#---main difference here. first you compute grads, and then clip them between thrshold and -thrshold before applying to the thetas
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)


with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.4902
1 Validation accuracy: 0.8384
2 Validation accuracy: 0.89
3 Validation accuracy: 0.9044
4 Validation accuracy: 0.9184
5 Validation accuracy: 0.927
6 Validation accuracy: 0.9308
7 Validation accuracy: 0.934
8 Validation accuracy: 0.9414
9 Validation accuracy: 0.9438
10 Validation accuracy: 0.943
11 Validation accuracy: 0.9482
12 Validation accuracy: 0.9516
13 Validation accuracy: 0.9544
14 Validation accuracy: 0.9552
15 Validation accuracy: 0.9584
16 Validation accuracy: 0.9586
17 Validation accuracy: 0.9612
18 Validation accuracy: 0.962
19 Validation accuracy: 0.9614


In [115]:
#here if we have the orginial code, we just restore the old model varables with restore and ccopy/edit the layers as we like

with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")



INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.964
1 Validation accuracy: 0.9604
2 Validation accuracy: 0.9658
3 Validation accuracy: 0.9662
4 Validation accuracy: 0.9672
5 Validation accuracy: 0.9678
6 Validation accuracy: 0.9678
7 Validation accuracy: 0.9684
8 Validation accuracy: 0.9698
9 Validation accuracy: 0.9716
10 Validation accuracy: 0.967
11 Validation accuracy: 0.9682
12 Validation accuracy: 0.972
13 Validation accuracy: 0.973
14 Validation accuracy: 0.9716
15 Validation accuracy: 0.9734
16 Validation accuracy: 0.9722
17 Validation accuracy: 0.9728
18 Validation accuracy: 0.973
19 Validation accuracy: 0.972


In [128]:
#----- just restoring some variables if we need selective layers------
import tensorflow as tf
import numpy as np


tf.reset_default_graph()

n_epochs = 20
batch_size = 200


(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]


n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
learning_rate = 0.01

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
  
#---we have two savers. one is a restore saver and the other to save the progress of this model
#the followig os all the graphkeys.global_variables
# [<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>, 
# <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>, 
# <tf.Variable 'hidden2/kernel:0' shape=(300, 50) dtype=float32_ref>, 
# <tf.Variable 'hidden2/bias:0' shape=(50,) dtype=float32_ref>, <tf.Variable 'hidden3/kernel:0' shape=(50, 50) dtype=float32_ref>, <tf.Variable 'hidden3/bias:0' shape=(50,) dtype=float32_ref>, <tf.Variable 'hidden4/kernel:0' shape=(50, 20) dtype=float32_ref>, <tf.Variable 'hidden4/bias:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'outputs/kernel:0' shape=(20, 10) dtype=float32_ref>, <tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>]

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
check = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
#     [print (a.name) for a in check]   #to check for the global variables
    for epoch in range(n_epochs):                                            # not shown in the book
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): # not shown
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})        # not shown
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})     # not shown
        print(epoch, "Validation accuracy:", accuracy_val)                   # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
hidden1/kernel:0
hidden1/bias:0
hidden2/kernel:0
hidden2/bias:0
hidden3/kernel:0
hidden3/bias:0
hidden4/kernel:0
hidden4/bias:0
outputs/kernel:0
outputs/bias:0


In [13]:
# Freezing the Lower Layers
tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 
n_hidden2 = 50  
n_hidden3 = 50  
n_hidden4 = 20  
n_outputs = 10  

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")


#---when freezing, we just tell the optmizer what variables to tweak when minimizing loss
#--- when training all layers, just remove the train_vars list 
with tf.name_scope("train"):                                        
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)    
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="hidden[34]|outputs")
    training_op = optimizer.minimize(loss, var_list=train_vars)

# global_variables inlclude information like following for all layers
#[<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>, 
#<tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>


reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")



INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.8412
1 Validation accuracy: 0.9186
2 Validation accuracy: 0.9332
3 Validation accuracy: 0.9376
4 Validation accuracy: 0.942
5 Validation accuracy: 0.944
6 Validation accuracy: 0.9462
7 Validation accuracy: 0.9476
8 Validation accuracy: 0.9498
9 Validation accuracy: 0.9498
10 Validation accuracy: 0.9502
11 Validation accuracy: 0.95
12 Validation accuracy: 0.9524
13 Validation accuracy: 0.9526
14 Validation accuracy: 0.9526
15 Validation accuracy: 0.9536
16 Validation accuracy: 0.9544
17 Validation accuracy: 0.9526
18 Validation accuracy: 0.9542
19 Validation accuracy: 0.955


In [23]:
tf.reset_default_graph()
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 
n_hidden2 = 50  
n_hidden3 = 50  
n_hidden4 = 20  
n_outputs = 10  

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name = "hidden1") 
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name = "hidden2")
    #this makes no difference when going forward. meanis hidden2 and hidden2_stop will be same. but when
    #computing gradients, input of this function will not be acted upon
    #also, if we dont use this, the statements go all the way to hidden1 and the code asks for value of X
    
    hidden2_stop = tf.stop_gradient(hidden2)
    
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation = tf.nn.relu, name = "hidden3") 
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation = tf.nn.relu, name = "hidden4")
    logits = tf.layers.dense(hidden4, n_outputs, name = "outputs") 

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") 
restore_saver = tf.train.Saver(reuse_vars)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter("logs/deepnetwork", tf.get_default_graph())

import numpy as np

n_batches = len(X_train) // batch_size

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    h2_cache = sess.run(hidden2, feed_dict={X: X_train})
    h2_cache_valid = sess.run(hidden2, feed_dict={X: X_valid}) 
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(len(X_train))
        hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(y_train[shuffled_idx], n_batches)
       
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={hidden2:hidden2_batch, y:y_batch})
        
        
        accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_valid, y: y_valid})            
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")



INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.8658
1 Validation accuracy: 0.9234
2 Validation accuracy: 0.9364
3 Validation accuracy: 0.9416
4 Validation accuracy: 0.9444
5 Validation accuracy: 0.945
6 Validation accuracy: 0.9464
7 Validation accuracy: 0.9476
8 Validation accuracy: 0.9486
9 Validation accuracy: 0.9496
10 Validation accuracy: 0.9504
11 Validation accuracy: 0.9512
12 Validation accuracy: 0.9508
13 Validation accuracy: 0.9504
14 Validation accuracy: 0.9512
15 Validation accuracy: 0.9516
16 Validation accuracy: 0.9526
17 Validation accuracy: 0.9498
18 Validation accuracy: 0.9518
19 Validation accuracy: 0.9532


In [24]:
#learning rate schedulings
tf.reset_default_graph()


n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

    
    #implementing lr(t) = lr(0)/10^-t/r
with tf.name_scope("train"):       
    initial_learning_rate = 0.1    #lr(0)
    decay_steps = 10000    #this is r
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")   #iteration number-t
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)
    #training_op will update the global step everytime by itself
    #global_step: Optional Variable to increment by one after the variables have been updated
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()


n_epochs = 5
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")



0 Validation accuracy: 0.9644
1 Validation accuracy: 0.9712
2 Validation accuracy: 0.9764
3 Validation accuracy: 0.9796
4 Validation accuracy: 0.9826


In [38]:
#l1 and l2 regularizations
#basically two new steps. 1) add it as the regulaizer when making layers and 2) add it to loss before gradients
tf.reset_default_graph()

from functools import partial

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

scale = 0.001

#without adding the kernel regualizer, best accuracy is 94.8 
my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None,
                            name="outputs")

with tf.name_scope("loss"):                                     
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)                                
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   #first this was loss. 
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")



0 Validation accuracy: 0.819
1 Validation accuracy: 0.8754
2 Validation accuracy: 0.8914
3 Validation accuracy: 0.8998
4 Validation accuracy: 0.9066
5 Validation accuracy: 0.911
6 Validation accuracy: 0.915
7 Validation accuracy: 0.9144
8 Validation accuracy: 0.9174
9 Validation accuracy: 0.9182
10 Validation accuracy: 0.9182
11 Validation accuracy: 0.9192
12 Validation accuracy: 0.9208
13 Validation accuracy: 0.9198
14 Validation accuracy: 0.9206
15 Validation accuracy: 0.9178
16 Validation accuracy: 0.92
17 Validation accuracy: 0.9192
18 Validation accuracy: 0.919
19 Validation accuracy: 0.919


In [41]:
#max-norm regularization
#here, we change the weights after they have been computed and updated
#in the one before, we were clipping gradients 

tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

threshold = 1.0
weights = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
clipped_weights = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights = tf.assign(weights, clipped_weights)

weights2 = tf.get_default_graph().get_tensor_by_name("hidden2/kernel:0")
clipped_weights2 = tf.clip_by_norm(weights2, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights2, clipped_weights2)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

with tf.Session() as sess:                                             
    init.run()                                                         
    for epoch in range(n_epochs):                                   
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights.eval()
            clip_weights2.eval()    
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid}) 
        print(epoch, "Validation accuracy:", acc_valid)                

    save_path = saver.save(sess, "./my_model_final.ckpt")     

0 Validation accuracy: 0.9594
1 Validation accuracy: 0.9674
2 Validation accuracy: 0.9764
3 Validation accuracy: 0.975
4 Validation accuracy: 0.9794
5 Validation accuracy: 0.9788
6 Validation accuracy: 0.9788
7 Validation accuracy: 0.9798
8 Validation accuracy: 0.9828
9 Validation accuracy: 0.9812
10 Validation accuracy: 0.9818
11 Validation accuracy: 0.9826
12 Validation accuracy: 0.9828
13 Validation accuracy: 0.982
14 Validation accuracy: 0.9834
15 Validation accuracy: 0.9822
16 Validation accuracy: 0.9834
17 Validation accuracy: 0.9832
18 Validation accuracy: 0.9828
19 Validation accuracy: 0.9834


In [149]:
#exercise questions --8

import tensorflow as tf
import warnings 
warnings.filterwarnings('ignore')
he_init = tf.variance_scaling_initializer()

import numpy as np
tf.reset_default_graph()

from functools import partial
n_inputs = 28 * 28  # MNIST


(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
#dataset for 0 1 2 3 4. 
X_train = X_train[y_train < 5]
X_test = X_test[y_test < 5]
y_train = y_train[y_train < 5]
y_test = y_test[y_test < 5]

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]


n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

my_hidden_layer = partial (tf.layers.dense, kernel_initializer=he_init, activation = tf.nn.elu)

with tf.name_scope("dnn"):
    hidden1 = my_hidden_layer(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = my_hidden_layer(hidden1, n_hidden2, name="hidden2")
    hidden3 = my_hidden_layer(hidden2, n_hidden3, name="hidden3")
    hidden4 = my_hidden_layer(hidden3, n_hidden4, name="hidden4")
    hidden5 = my_hidden_layer(hidden4, n_hidden5, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

#how they made layers. but this is just gonna output the result out of all the layers
# def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
#         activation=tf.nn.elu, initializer=he_init):
#     with tf.variable_scope(name, "dnn"):
#         for layer in range(n_hidden_layers):
#             inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
#                                      kernel_initializer=initializer,
#                                      name="hidden%d" % (layer + 1))
#         return inputs


# dnn_outputs = dnn(X)


with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()



def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

n_epochs = 40
batch_size = 50

max_checks_without_progress = 5
checks_without_progress = 0
best_loss = np.infty


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
#         print(epoch, "Batch loss:", loss_val, "Validation accuracy:", acc_val)

        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))
        
        

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))
#     save_path = saver.save(sess, "./my_model_final.ckpt")

0	Validation loss: 0.174616	Best loss: 0.174616	Accuracy: 96.72%
1	Validation loss: 0.086432	Best loss: 0.086432	Accuracy: 97.80%
2	Validation loss: 0.089369	Best loss: 0.086432	Accuracy: 97.92%
3	Validation loss: 0.144554	Best loss: 0.086432	Accuracy: 96.74%
4	Validation loss: 1.348296	Best loss: 0.086432	Accuracy: 34.94%
5	Validation loss: 0.426250	Best loss: 0.086432	Accuracy: 79.04%
6	Validation loss: 0.226934	Best loss: 0.086432	Accuracy: 96.86%
Early stopping!
INFO:tensorflow:Restoring parameters from ./my_mnist_model_0_to_4.ckpt
Final test accuracy: 98.46%


In [216]:
#####COPIED CODE

#classifierMixin has a default score func instead of fit_transform in transformixin
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        
        
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                        
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)



In [146]:
ym = [3,2,4,4,4,3,9,8,7,1]
classes = [ 1,4,4,4,7,8,9,9,3,2,3]
das = {label: index for index, label in enumerate(classes)}
yk = np.array([das[label] for label in ym], dtype=np.int32)
yk

array([10,  9,  3,  3,  3, 10,  7,  5,  4,  0], dtype=int32)

In [77]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train, y_train, n_epochs=1000, X_valid=X_valid, y_valid=y_valid)

0	Validation loss: 0.255437	Best loss: 0.255437	Accuracy: 95.56%
1	Validation loss: 0.128910	Best loss: 0.128910	Accuracy: 97.28%
2	Validation loss: 0.326685	Best loss: 0.128910	Accuracy: 91.52%
3	Validation loss: 0.224914	Best loss: 0.128910	Accuracy: 95.66%
4	Validation loss: 0.197522	Best loss: 0.128910	Accuracy: 94.92%
5	Validation loss: 0.146815	Best loss: 0.128910	Accuracy: 97.40%
6	Validation loss: 0.863341	Best loss: 0.128910	Accuracy: 65.40%
7	Validation loss: 0.324623	Best loss: 0.128910	Accuracy: 93.56%
8	Validation loss: 0.388992	Best loss: 0.128910	Accuracy: 92.04%
9	Validation loss: 0.118560	Best loss: 0.118560	Accuracy: 97.86%
10	Validation loss: 1.751891	Best loss: 0.118560	Accuracy: 41.10%
11	Validation loss: 1.175086	Best loss: 0.118560	Accuracy: 40.56%
12	Validation loss: 0.617364	Best loss: 0.118560	Accuracy: 73.50%
13	Validation loss: 0.528414	Best loss: 0.118560	Accuracy: 75.82%
14	Validation loss: 0.497790	Best loss: 0.118560	Accuracy: 77.84%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x7ff7abe1ed08>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [81]:
#predicting the test after making a simple DNN
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9840435882467407

In [80]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train, y_train, X_valid=X_valid, y_valid=y_valid, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params = dict(X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)
# rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                 fit_params=fit_params, random_state=42, verbose=2)
# rnd_search.fit(X_train1, y_train1)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x7ff7abe1ed08> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.156721	Best loss: 0.156721	Accuracy: 95.90%
1	Validation loss: 0.196882	Best loss: 0.156721	Accuracy: 94.54%
2	Validation loss: 0.190661	Best loss: 0.156721	Accuracy: 95.78%
3	Validation loss: 0.235254	Best loss: 0.156721	Accuracy: 95.04%
4	Validation loss: 0.162123	Best loss: 0.156721	Accuracy: 95.92%
5	Validation loss: 0.168211	Best loss: 0.156721	Accuracy: 95.72%
6	Validation loss: 0.169968	Best loss: 0.156721	Accuracy: 96.14%
7	Validation loss: 0.150931	Best loss: 0.150931	Accuracy: 95.92%
8	Validation loss: 0.145719	Best loss: 0.145719	Accuracy: 96.18%
9	Validation loss: 0.151230	Best loss: 0.145719	Accuracy: 96.36%
10	Validation loss: 0.149334	Best loss: 0.145719	Accuracy: 96.28%
11	Validation loss: 0.304815	Best loss: 0.145719	Accuracy: 91.88%
12	Validation loss: 0.175221	Best loss: 0.145719	Accuracy: 96.30%
13	Validation loss: 0.167786	Best loss: 0.145719	Accuracy: 94.94%
14	Validation loss: 0.132006	Best loss: 0.132006	Accuracy: 96.24%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.6s remaining:    0.0s


0	Validation loss: 0.165444	Best loss: 0.165444	Accuracy: 95.78%
1	Validation loss: 0.126509	Best loss: 0.126509	Accuracy: 96.52%
2	Validation loss: 0.169491	Best loss: 0.126509	Accuracy: 95.38%
3	Validation loss: 0.148133	Best loss: 0.126509	Accuracy: 96.42%
4	Validation loss: 0.128766	Best loss: 0.126509	Accuracy: 96.46%
5	Validation loss: 0.174608	Best loss: 0.126509	Accuracy: 96.60%
6	Validation loss: 0.166722	Best loss: 0.126509	Accuracy: 97.04%
7	Validation loss: 0.194460	Best loss: 0.126509	Accuracy: 96.50%
8	Validation loss: 0.158947	Best loss: 0.126509	Accuracy: 96.74%
9	Validation loss: 0.131404	Best loss: 0.126509	Accuracy: 96.96%
10	Validation loss: 0.143988	Best loss: 0.126509	Accuracy: 96.28%
11	Validation loss: 15.191450	Best loss: 0.126509	Accuracy: 32.44%
12	Validation loss: 1.626552	Best loss: 0.126509	Accuracy: 19.38%
13	Validation loss: 1.634317	Best loss: 0.126509	Accuracy: 19.24%
14	Validation loss: 1.614720	Best loss: 0.126509	Accuracy: 22.02%
15	Validation loss:

...

50	Validation loss: 0.306724	Best loss: 0.135262	Accuracy: 95.12%
51	Validation loss: 0.199052	Best loss: 0.135262	Accuracy: 96.56%
52	Validation loss: 0.147398	Best loss: 0.135262	Accuracy: 97.10%
53	Validation loss: 0.178707	Best loss: 0.135262	Accuracy: 96.74%
Early stopping!
[CV]  n_neurons=90, learning_rate=0.1, batch_size=500, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff70045a6a8>, total=  11.6s
[CV] n_neurons=90, learning_rate=0.1, batch_size=500, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff70045a6a8> 
0	Validation loss: 525.598206	Best loss: 525.598206	Accuracy: 41.92%
1	Validation loss: 28.255518	Best loss: 28.255518	Accuracy: 75.94%
2	Validation loss: 5.750528	Best loss: 5.750528	Accuracy: 92.14%
3	Validation loss: 1.941246	Best loss: 1.941246	Accuracy: 94.00%
4	Validation loss: 1.692047	Best loss: 1.692047	Accuracy: 93.80%
5	Validation loss: 1.612864	Best loss: 1.612864	Accuracy: 93.88%
6	Validation loss: 1.396625	Bes

25	Validation loss: 0.435725	Best loss: 0.435725	Accuracy: 97.20%
26	Validation loss: 0.677789	Best loss: 0.435725	Accuracy: 96.22%
27	Validation loss: 0.935618	Best loss: 0.435725	Accuracy: 95.40%
28	Validation loss: 0.926795	Best loss: 0.435725	Accuracy: 96.34%
29	Validation loss: 0.347182	Best loss: 0.347182	Accuracy: 97.14%
30	Validation loss: 0.364211	Best loss: 0.347182	Accuracy: 96.70%
31	Validation loss: 0.253266	Best loss: 0.253266	Accuracy: 97.42%
32	Validation loss: 0.642313	Best loss: 0.253266	Accuracy: 96.98%
33	Validation loss: 0.439407	Best loss: 0.253266	Accuracy: 96.56%
34	Validation loss: 0.345101	Best loss: 0.253266	Accuracy: 97.06%
35	Validation loss: 0.344751	Best loss: 0.253266	Accuracy: 96.98%
36	Validation loss: 0.357391	Best loss: 0.253266	Accuracy: 97.50%
37	Validation loss: 0.505138	Best loss: 0.253266	Accuracy: 96.46%
38	Validation loss: 0.406407	Best loss: 0.253266	Accuracy: 96.82%
39	Validation loss: 0.865290	Best loss: 0.253266	Accuracy: 96.50%
40	Validat

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 48.9min finished


0	Validation loss: 0.099962	Best loss: 0.099962	Accuracy: 97.08%
1	Validation loss: 0.069103	Best loss: 0.069103	Accuracy: 97.90%
2	Validation loss: 0.071453	Best loss: 0.069103	Accuracy: 98.00%
3	Validation loss: 0.079858	Best loss: 0.069103	Accuracy: 98.04%
4	Validation loss: 0.051466	Best loss: 0.051466	Accuracy: 98.50%
5	Validation loss: 0.080473	Best loss: 0.051466	Accuracy: 97.84%
6	Validation loss: 0.066668	Best loss: 0.051466	Accuracy: 98.46%
7	Validation loss: 0.074125	Best loss: 0.051466	Accuracy: 98.40%
8	Validation loss: 0.052180	Best loss: 0.051466	Accuracy: 98.58%
9	Validation loss: 0.045849	Best loss: 0.045849	Accuracy: 98.78%
10	Validation loss: 0.074447	Best loss: 0.045849	Accuracy: 98.44%
11	Validation loss: 0.052567	Best loss: 0.045849	Accuracy: 98.70%
12	Validation loss: 0.068658	Best loss: 0.045849	Accuracy: 98.50%
13	Validation loss: 0.051888	Best loss: 0.045849	Accuracy: 98.72%
14	Validation loss: 0.059053	Best loss: 0.045849	Accuracy: 98.62%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DNNClassifier(activation=<function elu at 0x7ff7abe1ed08>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid='warn', n_iter=50, n_jobs=None,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x7ff7abe00510>, <function elu at 0x7ff7abe1ed08>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff7007a7488>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff70045a6a8>]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          ret

In [87]:
#predicting on test set after tweaking hyperparameters using randomized search, no batch_normalization and dropout
from sklearn.metrics import accuracy_score
asd = rnd_search.best_estimator_

y_pred = asd.predict(X_test)
accuracy_score(y_test, y_pred)

# y_pred = rnd_search.predict(X_test1)
# accuracy_score(y_test1, y_pred)

rnd_search.best_estimator_.save("./my_best_mnist_model_0_to_4")

0.9898812998637867

In [96]:
#this DNNClassifier is our custom class that we are using to see how fast the algo is converging

dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                        n_neurons=140, random_state=42)
dnn_clf.fit(X_train, y_train, n_epochs=1000, X_valid=X_valid, y_valid=y_valid)



# y_pred = dnn_clf.predict(X_test)
# accuracy_score(y_test, y_pred)

0	Validation loss: 0.075695	Best loss: 0.075695	Accuracy: 97.64%
1	Validation loss: 0.062703	Best loss: 0.062703	Accuracy: 98.00%
2	Validation loss: 0.066334	Best loss: 0.062703	Accuracy: 98.02%
3	Validation loss: 0.056844	Best loss: 0.056844	Accuracy: 98.30%
4	Validation loss: 0.062461	Best loss: 0.056844	Accuracy: 98.36%
5	Validation loss: 0.057770	Best loss: 0.056844	Accuracy: 98.32%
6	Validation loss: 0.074312	Best loss: 0.056844	Accuracy: 98.28%
7	Validation loss: 0.071864	Best loss: 0.056844	Accuracy: 98.42%
8	Validation loss: 0.054139	Best loss: 0.054139	Accuracy: 98.62%
9	Validation loss: 0.058356	Best loss: 0.054139	Accuracy: 98.62%
10	Validation loss: 0.058746	Best loss: 0.054139	Accuracy: 98.68%
11	Validation loss: 0.060632	Best loss: 0.054139	Accuracy: 98.30%
12	Validation loss: 0.099874	Best loss: 0.054139	Accuracy: 97.70%
13	Validation loss: 0.061403	Best loss: 0.054139	Accuracy: 98.80%
14	Validation loss: 0.040204	Best loss: 0.040204	Accuracy: 98.94%
15	Validation loss: 

34	Validation loss: 0.262059	Best loss: 0.040204	Accuracy: 96.12%
35	Validation loss: 0.204411	Best loss: 0.040204	Accuracy: 96.46%
Early stopping!


DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff701267730>,
       batch_norm_momentum=None, batch_size=500, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=140,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [98]:
#trying with batch_normalization to see if it converges faster(it doesnt) but it does better the accuracy
dnn_clf_bn = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                           n_neurons=90, random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf_bn.fit(X_train, y_train, n_epochs=1000, X_valid=X_valid, y_valid=y_valid)


Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


0	Validation loss: 0.071428	Best loss: 0.071428	Accuracy: 97.88%
1	Validation loss: 0.057874	Best loss: 0.057874	Accuracy: 98.34%
2	Validation loss: 0.057646	Best loss: 0.057646	Accuracy: 98.36%
3	Validation loss: 0.045776	Best loss: 0.045776	Accuracy: 98.72%
4	Validation loss: 0.041274	Best loss: 0.041274	Accuracy: 98.72%
5	Validation loss: 0.057533	Best loss: 0.041274	Accuracy: 98.42%
6	Validation loss: 0.052958	Best loss: 0.041274	Accuracy: 98.48%
7	Validation loss: 0.051582	Best loss: 0.041274	Accuracy: 98.60%
8	Validation loss: 0.053446	Best loss: 0.041274	Accuracy: 98.68%
9	Validation loss: 0.044590	Best loss: 0.041274	Accuracy: 98.80%
10	Validation loss: 0.066788	Best loss: 0.041274	Accuracy: 98.36%
11	Validation loss: 0.038678	Best loss: 0.038678	Accuracy: 99.04%
12	Validation loss: 0.046524	Best loss: 0.038678	Accuracy: 98.66%
13	Validation loss: 0.045363	Best loss: 0.038678	Accuracy: 98.82%
14	Validation loss: 0.053328	Best loss: 0.038678	Accuracy: 98.68%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff6dca02b70>,
       batch_norm_momentum=0.95, batch_size=500, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=90,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [99]:
#testing accuracy with batch_normalization
y_pred = dnn_clf_bn.predict(X_test)

accuracy_score(y_test, y_pred)

0.9898812998637867

In [100]:
#using dropput of 0.5
dnn_clf_dropout = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                                n_neurons=90, random_state=42,
                                dropout_rate=0.5)
dnn_clf_dropout.fit(X_train, y_train, n_epochs=1000, X_valid=X_valid, y_valid=y_valid)

Instructions for updating:
Use keras.layers.dropout instead.


0	Validation loss: 0.131676	Best loss: 0.131676	Accuracy: 96.26%
1	Validation loss: 0.103470	Best loss: 0.103470	Accuracy: 96.94%
2	Validation loss: 0.097959	Best loss: 0.097959	Accuracy: 97.14%
3	Validation loss: 0.103814	Best loss: 0.097959	Accuracy: 97.18%
4	Validation loss: 0.090788	Best loss: 0.090788	Accuracy: 97.62%
5	Validation loss: 0.086620	Best loss: 0.086620	Accuracy: 97.74%
6	Validation loss: 0.082538	Best loss: 0.082538	Accuracy: 97.64%
7	Validation loss: 0.079264	Best loss: 0.079264	Accuracy: 97.88%
8	Validation loss: 0.086594	Best loss: 0.079264	Accuracy: 97.88%
9	Validation loss: 0.085434	Best loss: 0.079264	Accuracy: 97.94%
10	Validation loss: 0.075164	Best loss: 0.075164	Accuracy: 98.12%
11	Validation loss: 0.075030	Best loss: 0.075030	Accuracy: 98.20%
12	Validation loss: 0.091717	Best loss: 0.075030	Accuracy: 98.32%
13	Validation loss: 0.080773	Best loss: 0.075030	Accuracy: 98.02%
14	Validation loss: 0.081123	Best loss: 0.075030	Accuracy: 97.90%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff701267950>,
       batch_norm_momentum=None, batch_size=500, dropout_rate=0.5,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=90,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [102]:
#with dropput, accuracy has actually decreased
y_pred = dnn_clf_dropout.predict(X_test)
accuracy_score(y_test, y_pred)

0.9857949017318545

In [103]:
#tweaking parameters with dropout
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
    "dropout_rate": [0.2, 0.3, 0.4, 0.5, 0.6],
}

rnd_search_dropout = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=10,
                                        cv=3, random_state=42, verbose=2)
rnd_search_dropout.fit(X_train, y_train, X_valid=X_valid, y_valid=y_valid, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params = dict(X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)
# rnd_search_dropout = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                         fit_params=fit_params, random_state=42, verbose=2)
# rnd_search_dropout.fit(X_train1, y_train1)


#gives .9776 accuracy
# y_pred = rnd_search_dropout.predict(X_test)
# accuracy_score(y_test, y_pred)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=100, learning_rate=0.1, dropout_rate=0.5, batch_size=10, activation=<function elu at 0x7ff7abe1ed08> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 2.722634	Best loss: 2.722634	Accuracy: 19.24%
1	Validation loss: 2.563045	Best loss: 2.563045	Accuracy: 22.02%
2	Validation loss: 2.508170	Best loss: 2.508170	Accuracy: 19.38%
3	Validation loss: 2.153268	Best loss: 2.153268	Accuracy: 20.14%
4	Validation loss: 2.616973	Best loss: 2.153268	Accuracy: 22.02%
5	Validation loss: 2.507660	Best loss: 2.153268	Accuracy: 19.22%
6	Validation loss: 3.340463	Best loss: 2.153268	Accuracy: 20.14%
7	Validation loss: 1.981149	Best loss: 1.981149	Accuracy: 19.22%
8	Validation loss: 2.085649	Best loss: 1.981149	Accuracy: 19.22%
9	Validation loss: 2.300781	Best loss: 1.981149	Accuracy: 20.14%
10	Validation loss: 2.573446	Best loss: 1.981149	Accuracy: 19.38%
11	Validation loss: 2.254466	Best loss: 1.981149	Accuracy: 19.38%
12	Validation loss: 3.111515	Best loss: 1.981149	Accuracy: 19.24%
13	Validation loss: 2.095926	Best loss: 1.981149	Accuracy: 19.22%
14	Validation loss: 3.041722	Best loss: 1.981149	Accuracy: 19.38%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


0	Validation loss: 1.969009	Best loss: 1.969009	Accuracy: 19.24%
1	Validation loss: 2.546265	Best loss: 1.969009	Accuracy: 22.02%
2	Validation loss: 2.618414	Best loss: 1.969009	Accuracy: 22.02%
3	Validation loss: 2.280875	Best loss: 1.969009	Accuracy: 22.02%
4	Validation loss: 4.030173	Best loss: 1.969009	Accuracy: 19.24%
5	Validation loss: 2.904501	Best loss: 1.969009	Accuracy: 22.02%
6	Validation loss: 2.424104	Best loss: 1.969009	Accuracy: 19.38%
7	Validation loss: 3.686285	Best loss: 1.969009	Accuracy: 22.02%
8	Validation loss: 2.543800	Best loss: 1.969009	Accuracy: 22.02%
9	Validation loss: 3.281836	Best loss: 1.969009	Accuracy: 19.22%
10	Validation loss: 2.743168	Best loss: 1.969009	Accuracy: 19.24%
11	Validation loss: 2.128301	Best loss: 1.969009	Accuracy: 19.24%
12	Validation loss: 2.415043	Best loss: 1.969009	Accuracy: 19.22%
13	Validation loss: 2.654625	Best loss: 1.969009	Accuracy: 19.38%
14	Validation loss: 2.522216	Best loss: 1.969009	Accuracy: 22.02%
15	Validation loss: 

...

0	Validation loss: 0.432306	Best loss: 0.432306	Accuracy: 85.42%
1	Validation loss: 0.493826	Best loss: 0.432306	Accuracy: 89.38%
2	Validation loss: 0.908315	Best loss: 0.432306	Accuracy: 68.38%
3	Validation loss: 1.139715	Best loss: 0.432306	Accuracy: 60.00%
4	Validation loss: 1.990418	Best loss: 0.432306	Accuracy: 41.68%
5	Validation loss: 2.131732	Best loss: 0.432306	Accuracy: 31.98%
6	Validation loss: 0.912507	Best loss: 0.432306	Accuracy: 60.76%
7	Validation loss: 1.716327	Best loss: 0.432306	Accuracy: 48.52%
8	Validation loss: 2.487385	Best loss: 0.432306	Accuracy: 43.84%
9	Validation loss: 4.820521	Best loss: 0.432306	Accuracy: 31.64%
10	Validation loss: 9.104222	Best loss: 0.432306	Accuracy: 36.74%
11	Validation loss: 10.898355	Best loss: 0.432306	Accuracy: 39.16%
12	Validation loss: 10.439541	Best loss: 0.432306	Accuracy: 39.92%
13	Validation loss: 45.627151	Best loss: 0.432306	Accuracy: 19.24%
14	Validation loss: 7.404080	Best loss: 0.432306	Accuracy: 34.46%
15	Validation los

0	Validation loss: 0.854259	Best loss: 0.854259	Accuracy: 64.90%
1	Validation loss: 0.414729	Best loss: 0.414729	Accuracy: 89.70%
2	Validation loss: 0.298333	Best loss: 0.298333	Accuracy: 90.46%
3	Validation loss: 0.693487	Best loss: 0.298333	Accuracy: 73.02%
4	Validation loss: 0.745244	Best loss: 0.298333	Accuracy: 71.34%
5	Validation loss: 1.545331	Best loss: 0.298333	Accuracy: 54.36%
6	Validation loss: 1.276895	Best loss: 0.298333	Accuracy: 47.82%
7	Validation loss: 1.245020	Best loss: 0.298333	Accuracy: 61.56%
8	Validation loss: 1.620054	Best loss: 0.298333	Accuracy: 58.60%
9	Validation loss: 0.759556	Best loss: 0.298333	Accuracy: 71.02%
10	Validation loss: 1.053798	Best loss: 0.298333	Accuracy: 78.90%
11	Validation loss: 1.961670	Best loss: 0.298333	Accuracy: 44.68%
12	Validation loss: 1.346576	Best loss: 0.298333	Accuracy: 53.86%
13	Validation loss: 1.513185	Best loss: 0.298333	Accuracy: 48.44%
14	Validation loss: 1.642414	Best loss: 0.298333	Accuracy: 55.94%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 19.0min finished


0	Validation loss: 0.136186	Best loss: 0.136186	Accuracy: 97.00%
1	Validation loss: 0.154140	Best loss: 0.136186	Accuracy: 97.10%
2	Validation loss: 1.409114	Best loss: 0.136186	Accuracy: 35.14%
3	Validation loss: 1.735186	Best loss: 0.136186	Accuracy: 22.02%
4	Validation loss: 1.647532	Best loss: 0.136186	Accuracy: 20.14%
5	Validation loss: 1.658023	Best loss: 0.136186	Accuracy: 19.38%
6	Validation loss: 1.647885	Best loss: 0.136186	Accuracy: 19.22%
7	Validation loss: 1.647775	Best loss: 0.136186	Accuracy: 20.14%
8	Validation loss: 1.664379	Best loss: 0.136186	Accuracy: 19.24%
9	Validation loss: 1.655844	Best loss: 0.136186	Accuracy: 22.02%
10	Validation loss: 1.659307	Best loss: 0.136186	Accuracy: 22.02%
11	Validation loss: 1.616662	Best loss: 0.136186	Accuracy: 19.24%
12	Validation loss: 1.621090	Best loss: 0.136186	Accuracy: 22.02%
13	Validation loss: 1.643860	Best loss: 0.136186	Accuracy: 19.38%
14	Validation loss: 1.735983	Best loss: 0.136186	Accuracy: 19.22%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DNNClassifier(activation=<function elu at 0x7ff7abe1ed08>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x7ff7abe00510>, <function elu at 0x7ff7abe1ed08>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff6d79399d8>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7ff6d7939950>], 'dropout_rate': [0.2, 0.3, 0.4, 0.5, 0.6]},
          pre_dispatch='2*n_jobs',

In [192]:
#........question 9
tf.reset_default_graph()


(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X_train2_full = X_train[y_train >= 5]
y_train2_full = y_train[y_train >= 5] - 5
X_valid2_full = X_valid[y_valid >= 5]
y_valid2_full = y_valid[y_valid >= 5] - 5
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5] - 5


n_outputs = 5

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

learning_rate = 0.01

#<tf.Variable 'logits/bias:0' shape=(5,) dtype=float32_ref>] --- this is is the type returned by following
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)


init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [197]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)

    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    t0 = time.time()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.287019	Best loss: 1.287019	Accuracy: 49.33%
1	Validation loss: 1.060172	Best loss: 1.060172	Accuracy: 55.33%
2	Validation loss: 1.029877	Best loss: 1.029877	Accuracy: 61.33%
3	Validation loss: 0.966352	Best loss: 0.966352	Accuracy: 58.67%
4	Validation loss: 1.000900	Best loss: 0.966352	Accuracy: 56.67%
5	Validation loss: 1.017273	Best loss: 0.966352	Accuracy: 58.00%
6	Validation loss: 0.926260	Best loss: 0.926260	Accuracy: 59.33%
7	Validation loss: 1.042750	Best loss: 0.926260	Accuracy: 52.67%
8	Validation loss: 0.913795	Best loss: 0.913795	Accuracy: 64.00%
9	Validation loss: 0.994793	Best loss: 0.913795	Accuracy: 58.67%
10	Validation loss: 0.940518	Best loss: 0.913795	Accuracy: 60.67%
11	Validation loss: 1.024206	Best loss: 0.913795	Accuracy: 60.00%
12	Validation loss: 0.928713	Best loss: 0.913795	Accuracy: 65.33%
13	Validation loss: 0.928180	Best loss: 0.913795	Accuracy: 62.00%
14	Validation l

In [200]:
#the following statement gets the who tensor equation i think
hidden5_out = tf.get_default_graph().get_tensor_by_name("hidden5_out:0")

import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    t0 = time.time()
    
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.213265	Best loss: 1.213265	Accuracy: 50.00%
1	Validation loss: 1.027073	Best loss: 1.027073	Accuracy: 55.33%
2	Validation loss: 1.007629	Best loss: 1.007629	Accuracy: 58.00%
3	Validation loss: 0.973284	Best loss: 0.973284	Accuracy: 58.67%
4	Validation loss: 1.001585	Best loss: 0.973284	Accuracy: 62.00%
5	Validation loss: 0.967088	Best loss: 0.967088	Accuracy: 60.67%
6	Validation loss: 1.024953	Best loss: 0.967088	Accuracy: 58.00%
7	Validation loss: 0.955386	Best loss: 0.955386	Accuracy: 58.67%
8	Validation loss: 0.945408	Best loss: 0.945408	Accuracy: 60.67%
9	Validation loss: 1.003105	Best loss: 0.945408	Accuracy: 58.00%
10	Validation loss: 0.907099	Best loss: 0.907099	Accuracy: 68.00%
11	Validation loss: 0.924137	Best loss: 0.907099	Accuracy: 64.00%
12	Validation loss: 0.941346	Best loss: 0.907099	Accuracy: 60.67%
13	Validation loss: 0.966442	Best loss: 0.907099	Accuracy: 59.33%
14	Validation l

In [201]:
# Exercise: try again reusing just four hidden layers instead of five. Can you achieve a higher precision?
tf.reset_default_graph()

n_outputs = 5

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("hidden4_out:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_5_to_9_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.076254	Best loss: 1.076254	Accuracy: 54.00%
1	Validation loss: 0.995773	Best loss: 0.995773	Accuracy: 59.33%
2	Validation loss: 0.976076	Best loss: 0.976076	Accuracy: 56.67%
3	Validation loss: 0.966688	Best loss: 0.966688	Accuracy: 62.67%
4	Validation loss: 0.943388	Best loss: 0.943388	Accuracy: 63.33%
5	Validation loss: 0.867194	Best loss: 0.867194	Accuracy: 66.67%
6	Validation loss: 0.888369	Best loss: 0.867194	Accuracy: 65.33%
7	Validation loss: 0.854243	Best loss: 0.854243	Accuracy: 64.00%
8	Validation loss: 0.871319	Best loss: 0.854243	Accuracy: 64.00%
9	Validation loss: 0.889557	Best loss: 0.854243	Accuracy: 63.33%
10	Validation loss: 0.875299	Best loss: 0.854243	Accuracy: 65.33%
11	Validation loss: 0.825011	Best loss: 0.825011	Accuracy: 67.33%
12	Validation loss: 0.827345	Best loss: 0.825011	Accuracy: 66.67%
13	Validation loss: 0.785680	Best loss: 0.785680	Accuracy: 73.33%
14	Validation l

In [204]:
#unfreezing all layers and restoring from the old saver
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam4")
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
no_frozen_saver = tf.train.Saver()

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = no_frozen_saver.save(sess, "./my_mnist_model_5_to_9_no_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    no_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_no_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
0	Validation loss: 0.810452	Best loss: 0.810452	Accuracy: 78.00%
1	Validation loss: 0.691513	Best loss: 0.691513	Accuracy: 84.00%
2	Validation loss: 0.733665	Best loss: 0.691513	Accuracy: 89.33%
3	Validation loss: 0.534370	Best loss: 0.534370	Accuracy: 91.33%
4	Validation loss: 0.627864	Best loss: 0.534370	Accuracy: 89.33%
5	Validation loss: 0.609417	Best loss: 0.534370	Accuracy: 92.67%
6	Validation loss: 0.716454	Best loss: 0.534370	Accuracy: 92.00%
7	Validation loss: 0.542284	Best loss: 0.534370	Accuracy: 91.33%
8	Validation loss: 0.450488	Best loss: 0.450488	Accuracy: 92.67%
9	Validation loss: 1.620248	Best loss: 0.450488	Accuracy: 84.67%
10	Validation loss: 0.594070	Best loss: 0.450488	Accuracy: 94.00%
11	Validation loss: 1.103480	Best loss: 0.450488	Accuracy: 92.00%
12	Validation loss: 0.696126	Best loss: 0.450488	Accuracy: 92.67%
13	Validation loss: 0.954892	Best loss: 0.450488	Accuracy: 88.67%
14	Valid

In [205]:
#compared to what we did above, if we made a DNN from scratch, we get this accuracy
dnn_clf_5_to_9 = DNNClassifier(n_hidden_layers=4, random_state=42)
dnn_clf_5_to_9.fit(X_train2, y_train2, n_epochs=1000, X_valid=X_valid2, y_valid=y_valid2)

# y_pred = dnn_clf_5_to_9.predict(X_test2)
# accuracy_score(y_test2, y_pred)


0	Validation loss: 0.674608	Best loss: 0.674608	Accuracy: 80.67%
1	Validation loss: 0.584855	Best loss: 0.584855	Accuracy: 88.67%
2	Validation loss: 0.647364	Best loss: 0.584855	Accuracy: 84.00%
3	Validation loss: 0.530303	Best loss: 0.530303	Accuracy: 87.33%
4	Validation loss: 0.683543	Best loss: 0.530303	Accuracy: 90.67%
5	Validation loss: 0.534674	Best loss: 0.530303	Accuracy: 89.33%
6	Validation loss: 0.712317	Best loss: 0.530303	Accuracy: 90.67%
7	Validation loss: 1.032102	Best loss: 0.530303	Accuracy: 84.67%
8	Validation loss: 0.693217	Best loss: 0.530303	Accuracy: 90.00%
9	Validation loss: 0.337214	Best loss: 0.337214	Accuracy: 91.33%
10	Validation loss: 0.708275	Best loss: 0.337214	Accuracy: 90.67%
11	Validation loss: 1.186233	Best loss: 0.337214	Accuracy: 87.33%
12	Validation loss: 0.695534	Best loss: 0.337214	Accuracy: 93.33%
13	Validation loss: 1.676602	Best loss: 0.337214	Accuracy: 86.67%
14	Validation loss: 0.873113	Best loss: 0.337214	Accuracy: 90.67%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x7ff7abe1ed08>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7ff7037e1978>,
       learning_rate=0.01, n_hidden_layers=4, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [206]:
y_pred = dnn_clf_5_to_9.predict(X_test2)
accuracy_score(y_test2, y_pred)

0.8987862579716108

In [217]:
def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs


#question 10

n_inputs = 28 * 28 # MNIST

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, 2, n_inputs), name="X")
X1, X2 = tf.unstack(X, axis=1)

y = tf.placeholder(tf.int32, shape=[None, 1])

dnn1 = dnn(X1, name="DNN_A")
dnn2 = dnn(X2, name="DNN_B")

dnn_outputs = tf.concat([dnn1, dnn2], axis=1)


hidden = tf.layers.dense(dnn_outputs, units=10, activation=tf.nn.elu, kernel_initializer=he_init)
logits = tf.layers.dense(hidden, units=1, kernel_initializer=he_init)
y_proba = tf.nn.sigmoid(logits)

y_pred = tf.cast(tf.greater_equal(logits, 0), tf.int32)

y_as_float = tf.cast(y, tf.float32)
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_as_float, logits=logits)
loss = tf.reduce_mean(xentropy)

learning_rate = 0.01
momentum = 0.95

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

y_pred_correct = tf.equal(y_pred, y)
accuracy = tf.reduce_mean(tf.cast(y_pred_correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [225]:
X_train1 = X_train
y_train1 = y_train

X_train2 = X_valid
y_train2 = y_valid

X_test = X_test
y_test = y_test

def generate_batch(images, labels, batch_size):
    size1 = batch_size // 2
    size2 = batch_size - size1
    if size1 != size2 and np.random.rand() > 0.5:
        size1, size2 = size2, size1
    X = []
    y = []
    while len(X) < size1:
        rnd_idx1, rnd_idx2 = np.random.randint(0, len(images), 2)
        #-----------------same number-----------------
        if rnd_idx1 != rnd_idx2 and labels[rnd_idx1] == labels[rnd_idx2]:
            X.append(np.array([images[rnd_idx1], images[rnd_idx2]]))
            y.append([1])
    while len(X) < batch_size:
        rnd_idx1, rnd_idx2 = np.random.randint(0, len(images), 2)
        #---------------different number
        if labels[rnd_idx1] != labels[rnd_idx2]:
            X.append(np.array([images[rnd_idx1], images[rnd_idx2]]))
            y.append([0])
    rnd_indices = np.random.permutation(batch_size)
    return np.array(X)[rnd_indices], np.array(y)[rnd_indices]

batch_size = 5
X_batch, y_batch = generate_batch(X_train1, y_train1, batch_size)

X_batch.shape, X_batch.dtype

((5, 2, 784), dtype('float32'))

In [226]:

X_test1, y_test1 = generate_batch(X_test, y_test, batch_size=len(X_test))

n_epochs = 100
batch_size = 500

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(X_train1) // batch_size):
            X_batch, y_batch = generate_batch(X_train1, y_train1, batch_size)
            loss_val, _ = sess.run([loss, training_op], feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train loss:", loss_val)
        if epoch % 5 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
            print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_digit_comparison_model.ckpt")

0 Train loss: 0.6903936
0 Test accuracy: 0.5082
1 Train loss: 0.6833481
2 Train loss: 0.59663993
3 Train loss: 0.52677065
4 Train loss: 0.45733145
5 Train loss: 0.44001445
5 Test accuracy: 0.8233
6 Train loss: 0.36042613
7 Train loss: 0.320972
8 Train loss: 0.29581523
9 Train loss: 0.2526048
10 Train loss: 0.28619477
10 Test accuracy: 0.882
11 Train loss: 0.20689356
12 Train loss: 0.21096022
13 Train loss: 0.22925885
14 Train loss: 0.23902874
15 Train loss: 0.22255908
15 Test accuracy: 0.9137
16 Train loss: 0.16367
17 Train loss: 0.23098688
18 Train loss: 0.18356904
19 Train loss: 0.17443101
20 Train loss: 0.16559376
20 Test accuracy: 0.9353
21 Train loss: 0.17468473
22 Train loss: 0.14474645
23 Train loss: 0.14061904
24 Train loss: 0.15962403
25 Train loss: 0.15495135
25 Test accuracy: 0.9511
26 Train loss: 0.13098834
27 Train loss: 0.09978789
28 Train loss: 0.1143946
29 Train loss: 0.12438205
30 Train loss: 0.10428523
30 Test accuracy: 0.9568
31 Train loss: 0.07902643
32 Train loss: 

In [235]:
tf.reset_default_graph()
n_inputs = 28 * 28  # MNIST
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X, name="DNN_A")
frozen_outputs = tf.stop_gradient(dnn_outputs)

logits = tf.layers.dense(frozen_outputs, n_outputs, kernel_initializer=he_init)
Y_proba = tf.nn.softmax(logits)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

check =  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="DNN_A")
dnn_A_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="DNN_A")
# var_list specifies the variables that will be saved and restored. It can be passed as a dict or a list:
# A dict of names to variables: The keys are the names that will be used to save or restore 
#the variables in the checkpoint files. A list of variables: The variables will be keyed with their op name in the checkpoint files    
restore_saver = tf.train.Saver(var_list={var.op.name: var for var in dnn_A_vars})
saver = tf.train.Saver()

n_epochs = 100
batch_size = 50

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_digit_comparison_model.ckpt")

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 10 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_mnist_model_final.ckpt")


INFO:tensorflow:Restoring parameters from ./my_digit_comparison_model.ckpt
0 Test accuracy: 0.9702
10 Test accuracy: 0.9733
20 Test accuracy: 0.9738
30 Test accuracy: 0.9746
40 Test accuracy: 0.9749
50 Test accuracy: 0.9742
60 Test accuracy: 0.9733
70 Test accuracy: 0.9745
80 Test accuracy: 0.9745
90 Test accuracy: 0.9738


In [236]:
#if there were no transfer learning

tf.reset_default_graph()
n_inputs = 28 * 28  # MNIST
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X, name="DNN_A")

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init)
Y_proba = tf.nn.softmax(logits)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

dnn_A_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="DNN_A")
restore_saver = tf.train.Saver(var_list={var.op.name: var for var in dnn_A_vars})
saver = tf.train.Saver()

n_epochs = 150
batch_size = 50

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 10 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_mnist_model_final.ckpt")

0 Test accuracy: 0.8775
10 Test accuracy: 0.934
20 Test accuracy: 0.9221
30 Test accuracy: 0.9356
40 Test accuracy: 0.9418
50 Test accuracy: 0.9431
60 Test accuracy: 0.9432
70 Test accuracy: 0.9435
80 Test accuracy: 0.9431
90 Test accuracy: 0.943
100 Test accuracy: 0.9428
110 Test accuracy: 0.9429
120 Test accuracy: 0.9429
130 Test accuracy: 0.9427
140 Test accuracy: 0.9427
